In [39]:
import pandas as pd
import glob

## Data loading

In [40]:


dfImages = pd.read_csv('../data/IAPS.csv',
                       sep=';',
                       usecols=['IAPS', 'ValenceMean', 'ArousalMean']
                      )



dfSounds = pd.read_csv('../data/IADS2.csv', sep=';', usecols=['Number', 'ValenceMean', 'ArousalMean'])


studyDatafiles = glob.glob('../data/procedura/*.txt')

li = []

for filename in studyDatafiles:
    df = pd.read_csv(filename, sep='\t', header=None, names=['ID', 'nn', 'Condition', 'SpecificCondition', 'Number', 'IAPS', 'Widget', 'Response', 'Delay', 'ApperanceTimestamp'])
    li.append(df)
                         
dfStudy = pd.concat(li, axis=0, ignore_index=True)

dfStudy = dfStudy[dfStudy.Condition != 'con']
dfStudy = dfStudy[dfStudy.Widget != 'emoscale1']
                                     


## Preprocessing

In [41]:


dfStudy['Number'] = pd.to_numeric(dfStudy['Number'])
dfStudy['IAPS'] = pd.to_numeric(dfStudy['IAPS'])



def convertResponseStringIntoColumns(string, i):
    string = string[1:-1]
    array = (string.split(','))
    array[0] = float(array[0])
    array[1] = float(array[1][1:])
    return array[i]
    
    
dfStudy['Valence'] = dfStudy.Response.apply(func = convertResponseStringIntoColumns, i = 0)
dfStudy['Arousal'] = dfStudy.Response.apply(func = convertResponseStringIntoColumns, i = 1)

dfStudy = dfStudy[['SpecificCondition', 'Number', 'IAPS', 'Valence', 'Arousal']]

dfImages
dfSounds
dfStudy

             

,SpecificCondition,Number,IAPS,Valence,Arousal
0,p-s+,215,3101,0.084368,0.157365
3,p+s-,260,4490,-0.431722,0.192667
4,p+s-,277,4647,-0.077128,0.272818
6,p+s-,422,4800,0.008572,0.390008
7,p+s-,290,4697,-0.300478,0.363130
9,p-s+,204,3053,-0.458174,0.384430
12,p+s-,288,4668,-0.099542,0.326573
16,p+s-,289,8186,0.000000,0.218752
18,p-s+,351,6250,0.000000,0.136890
19,p-s+,601,3195,-0.098290,0.170638
